In [20]:
#텍스트 마이닝
#2025.04.10 ~ 2025.05. 진행

#############################

#[진행목적]
#네이버에 키워드 '인제' 검색 후 나오는 4개년치 데이터 웹크롤링 진행
#웹크롤링을 통해 강원도 지역 인제와 인제 자작나무숲에 대한 트렌드 및 인식 파악


#[순서]
#빈도(워드클라우드 시각화) - TF-IDF - n-gram - 토픽모델링(LDA / bertpic) - concor분석(UCINET시각화) - 감정분석


#[참고]
#보안상 원본데이터 반출이 금지되어, 실제 데이터와 결과는 비슷 하지만 추론이 불가능한 가데이터로 다시 만든 코드
#실제 연구에서는 4개년치의 데이터를 사용했지만 데이터만 다를뿐 동일한 방식으로 연구를 진행하여 가데이터는 1년치 데이터만 만들어 진행
#데이터만 다를 뿐 실제 연구에 사용된 코드와 동일 코드 

In [21]:
##빈도 분석##

#빈도는 명사, 동사, 형용사만 추출 예정
#1~100위 까지 추출

In [22]:
#필요한 패키지 로드

import pandas as pd
import re
import emoji
from tqdm.notebook import tqdm
from collections import Counter
from konlpy.tag import Okt
from soynlp.normalizer import *
from kiwipiepy import Kiwi

In [23]:
#데이터 준비(가데이터 생성)

import pandas as pd
import random

# 실제 원본데이터 결과와 비슷한 개수로 주제별 게시글 수 지정
post_counts = {
    "강원도 인제": 76,
    "인제 맛집": 67,
    "인제 가족여행": 37,
    "강원도 속초": 36,
    "강원도 춘천": 27,
    "광고 게시글": 29,
    "인제 카페": 28
}

# 실제 데이터와 유사한 주제로 가짜 블로그 게시글 생성
sentence_templates = {
    "강원도 인제": [
        "강원도 인제 진짜 너무 좋았어요 ㅎㅎ",
        "자작나무 숲..사진보다 실물이 훨씬 예뻐요!",
        "산책하면서 마음이 정화되는 기분이랄까~",
        "맑은 공기 마시니까 기분까지 맑아지더라구요~ㅎㅎ",
        "드디어 포스팅해보는 인제여행기ㅋㅋㅋ 아 또가고싶다",
        "역시 자작나무 숲은 여름보단 겨울이 이쁜듯",
        '원대리 자작나무 숲 여행기 지금 시작합니다!',
        '오늘 소개할 여행지는 인제 원대리 자작나무 숲!',
        '벼르고 벼르다 드디어 떠난 인제 여행..눈물난다ㅠㅠ',
        "여긴 진짜 힐링하러 가기 딱이에요! 강추!",
        "난 언제 여행가나 했는데 드디어 왔습니다 강원도 인제..",
        "ㅋㅋㅋㅋ남친이 좀 무리하긴 했지만 서로 연차 맞춰서 드디어 인제로 떠남",
        "여행기념 사진도 하나 찍고~",
        "완전 가는 곳곳마다 사진 맛집ㅋㅋ인생사진 겁나 건짐",
        "오랜만에 숲길 걸으니 너무상쾌해서 좋았다.",
        "역시 사람은 도시가 아니라 자연속에 있어야해..매연없는 공기가 이렇게 좋다니",
        "성수기에 와서 그런지 걱정했던것보단 아니지만 그래도 사람들이 꽤 많았음. 다들 여름 휴가는 인제에 오기로 약속한거야?",
        "도착하자마자 너무 배고파서 길거리 음식도 하나 먹고"
    ],
    "인제 맛집": [
        "인제에 이런 숨은 황태 맛집이?! 진짜 대박이에요 ㅋㅋ",
        "손두부 맛집 남편하고 맛있게 먹음. 엄청 고소하고 주인 할머니가 친절해서 좋았어요~",
        "인제에 왔으면 막국수를 먹어야지 인터넷에서 후기보고왔는데 역시..대 존 맛",
        "인제 황태 유명한건 다들 아시죠~ 저도 아들이랑 먹으러 왔는데 너무 좋아해요~",
        "현지인 찐추천 받아서 갔는데 성공적✨ \n 존재하지 않는 스티커입니다",
        "인제 고기 육즙 무엇..? 입에서 살살 녹음ㅋㅋㅋ 생각했던것보다 더 많이 먹어서 스스로도 놀랬지만 기왕여행온거 잘먹어야하는것",
        "국밥 한그릇에 감동받기 있음? 여긴 가능;;",
        "비 오는 날 막국수랑 전 조합은 진리죠ㅠㅠ",
        "너무 유명한 이제 손두부 맛집. 가격도 착하고 맛도 너무 괜찮아서 다음에 여행오게 된다면 여기로 또 오고싶을 정도에요!",
        "하 역시 황태랑 막국수는 먹어야함. 친구랑 막국수 2인분시켜서 다 먹고 낮이긴 하지만~ 맥주도 시켜서 시원하게 즐겨줬음ㅋㅋ",
    ],
    "인제 가족여행": [
        "가족끼리 인제 자작나무숲 다녀왔는데 애들이 너무 좋아했어요~",
        "겨울이 유명하다는 인제 자작나무 숲 역시 눈 쌓인 숲이 너무 예쁨ㅠ",
        "아빠가 완전 좋아했음ㅋㅋ가족끼리 놀러오기 딱 좋은듯 또 가고싶다",
        "오랜만에 \n 엄마랑 둘이 떠난 인제 여행. 원대리 자작나무 숲으로~! \n 존재하지 않는 이미지입니다",
        "늦은밤 퇴근 후 인제로 바로 고고..피곤했지만 그래도 오랜만에 떠나는 여행인 만큼 웃으며 떠났음",
        "차박하기도 좋고 캠핑하기도 좋은듯",
        "캠핑좋아하는 아내가 하도 말하길래 왔는데 캠핑별로 안좋아하는 나도 너무 잘 놀았음",
        "숙소도 깔끔하고 근처에 놀 곳도 많아서 완전 만족~!",
        "아 재밌었다 진짜ㅋㅋ",
        "또 가고싶다",
        "놀이공원은 없지만 자연이 주는 재미가 있어요 ㅎㅎ",
        "애들하고 캠핑했는데 너무 즐거웠던 하루였어요.",
        "다음에도 또 가자고 졸라요ㅋㅋㅋ"
    ],
    "강원도 속초": [
        "강원도 속초 바다는 언제 가도 좋아요🌊",
        "파도소리들으며 잠시 힐링 \n 존재하지 않는 이미지 입니다",
        "파도치는 바다는 멍하게 보게되는 무언가가 있는거같아",
        "시장 구경하다가 오징어 순대 먹었는데 존맛ㅠㅠ",
        "속초해수욕장 산책하다가 일몰에 감동...",
        "언제 또 속초를 가보겠어..이것저것 다먹어야지",
        "완전 힐링여행 그자체 \n 존재하지 않는 이미지입니다 \nㅎㅎㅎㅎ",
        "설악산 케이블카 타고 위에서 내려다본 풍경 진짜 최고였어요!",
        "속초엔 \n 맛집이랑 카페도 많아서 하루가 금방 가요 ㅎㅎㅎ"
        "피곤하긴했지만 즐거웠던 여행",
        "속도 바다바람 너무 시원해",
        "역시 사람은 바다를 좀 보고 살아야해",
        "오랜만에 탁 트인 바다를 보니 기분이 너무 좋아졌음 \n존재하지 않는 스티커입니다"
    ],
    "강원도 춘천": [
        "춘천 하면 닭갈비지!! 진짜 맛있게 먹었어요 ㅋㅋ",
        "예전부터 강원도 춘천으로 떠나는 상상을 하며 지냈는데 \n 드디어 오늘 출발한다",
        "강 따라 자전거 타는 코스도 좋아요~",
        "춘천 카페거리 감성 터져요📸",
        "막찍어도 인생사진 그냥 건짐",
        "너무 즐거운 여행 \n",
        "애견 동반 가능한 장소도 꽤 있더라구요~! \n 존재하지 않는 이미지입니다",
        "강원도 춘천에서 사진찍고 바로 그 옆에 있는 막국수 & 닭갈비 집으로",
        "춘천역 앞에서 사진도 찍고~ \n 기차여행 느낌 물씬ㅎㅎㅎ"
    ],
    "광고 게시글": [
        "📢 지금 가입만 해도 100% 에어컨 당첨 혜택🎉 지금 에어컨 받아가세요",
        "✔️키즈 장난감 인기 상품 최저가 보장! 지금 바로 클릭",
        "통신사 단 3일간! 특가 이벤트 중입니다~ 서두르세요! 할인 기다려주신분들 할인혜택은 인제부터 시작입니다!",
        "이런 기회 인제 다시 없어요 ㅋㅋㅋ 물광 마스크 팩 지금 주문하면 1+1",
        "무선 선풍기 무료체험 신청하면 사은품도 드려요😍 당첨기회",
        "인제 무더운 날씨가 성큼 찾아왔어요ㅠ \n 우리아이 피부도 예민한데 쿨링패드 하나면 충분해요~",
        "영어학원 어디 찾으세요? 영어학원 고민은 인제 그만! \n 우리아이 초등교육 책임질 스피키 영어 학원",
        "이 혜택이면 고민 끝이죠? 지금 바로 주문고고!"
    ],
    "인제 카페": [
        "인제에도 요즘 핫한 카페가 생겼어요~ 분위기 깡패!!",
        "인제 커피 맛집 찾았어요ㅠㅠ 진짜 고소하고 맛있어요ㅎㅎㅎ",
        "디저트가 비주얼 끝판왕... 인스타 감성 뿜뿜✨",
        "카페 분위기 너무 좋아",
        "여기는 디저트가 다했다 진짜",
        "당근 케이크 꼭 드세요",
        "시그니쳐 메뉴가 진짜 맛있음",
        "그냥 평범한 아메리카노인데 왜이렇게 맛있니",
        "디저트 다 맛있는데 강아지랑 같이 있을 수 있어서 더 좋은듯ㅋㅋㅋ",
        "인제 애견카페 추천 강아지랑 즐거운 시간 보내기 조아용",
        "인제 마당있는 카페라서 산책하기 좋음",
        "커플끼리, 친구 끼리, 가족끼리 그냥 오기도 너무 좋은 분위기였음",
        "창밖 보면서 멍 때리기 딱 좋은 카페였어요~",
        "바닐라라떼랑 조각케이크 조합은 언제나 옳죠😋 \n 존재하지 않는 이미지입니다",
        "인제 갬성 오지는 카페에 드디어 왔다규..커피도 존맛이라구",
        "사실 큰 기대 안하고 주문한 시그니처 초콜릿 라떼가 너무 맛있어서 깜짝놀람"
    ]
}

# 게시글 생성 함수
def create_blog_post(topic):
    lines = random.choices(sentence_templates[topic], k=random.randint(5, 7))
    text = "\n".join(lines[:random.randint(2, 4)]) + "\n\n" + "\n".join(lines[random.randint(2, 4):])
    return text

# 전체 데이터 생성
data = []
for topic, count in post_counts.items():
    for _ in range(count):
        post_text = create_blog_post(topic)
        data.append({"text": post_text})

df = pd.DataFrame(data)

In [24]:
#생성된 데이터 확인

df['text']

0      난 언제 여행가나 했는데 드디어 왔습니다 강원도 인제..\n오랜만에 숲길 걸으니 너...
1      난 언제 여행가나 했는데 드디어 왔습니다 강원도 인제..\n성수기에 와서 그런지 걱...
2      역시 사람은 도시가 아니라 자연속에 있어야해..매연없는 공기가 이렇게 좋다니\n드디...
3      벼르고 벼르다 드디어 떠난 인제 여행..눈물난다ㅠㅠ\n원대리 자작나무 숲 여행기 지...
4      오랜만에 숲길 걸으니 너무상쾌해서 좋았다.\n산책하면서 마음이 정화되는 기분이랄까~...
                             ...                        
295    인제에도 요즘 핫한 카페가 생겼어요~ 분위기 깡패!!\n인제 마당있는 카페라서 산책...
296    인제 갬성 오지는 카페에 드디어 왔다규..커피도 존맛이라구\n커플끼리, 친구 끼리,...
297    당근 케이크 꼭 드세요\n사실 큰 기대 안하고 주문한 시그니처 초콜릿 라떼가 너무 ...
298    사실 큰 기대 안하고 주문한 시그니처 초콜릿 라떼가 너무 맛있어서 깜짝놀람\n디저트...
299    인제 마당있는 카페라서 산책하기 좋음\n인제 마당있는 카페라서 산책하기 좋음\n인제...
Name: text, Length: 300, dtype: object

In [25]:
#df를 리스트화 만들기
post_li = df['text'].to_list()


print(post_li[:5])

['난 언제 여행가나 했는데 드디어 왔습니다 강원도 인제..\n오랜만에 숲길 걸으니 너무상쾌해서 좋았다.\n드디어 포스팅해보는 인제여행기ㅋㅋㅋ 아 또가고싶다\n\n맑은 공기 마시니까 기분까지 맑아지더라구요~ㅎㅎ', '난 언제 여행가나 했는데 드디어 왔습니다 강원도 인제..\n성수기에 와서 그런지 걱정했던것보단 아니지만 그래도 사람들이 꽤 많았음. 다들 여름 휴가는 인제에 오기로 약속한거야?\n\n완전 가는 곳곳마다 사진 맛집ㅋㅋ인생사진 겁나 건짐', '역시 사람은 도시가 아니라 자연속에 있어야해..매연없는 공기가 이렇게 좋다니\n드디어 포스팅해보는 인제여행기ㅋㅋㅋ 아 또가고싶다\n성수기에 와서 그런지 걱정했던것보단 아니지만 그래도 사람들이 꽤 많았음. 다들 여름 휴가는 인제에 오기로 약속한거야?\n자작나무 숲..사진보다 실물이 훨씬 예뻐요!\n\n자작나무 숲..사진보다 실물이 훨씬 예뻐요!\n성수기에 와서 그런지 걱정했던것보단 아니지만 그래도 사람들이 꽤 많았음. 다들 여름 휴가는 인제에 오기로 약속한거야?\n벼르고 벼르다 드디어 떠난 인제 여행..눈물난다ㅠㅠ\n여긴 진짜 힐링하러 가기 딱이에요! 강추!', '벼르고 벼르다 드디어 떠난 인제 여행..눈물난다ㅠㅠ\n원대리 자작나무 숲 여행기 지금 시작합니다!\n\n도착하자마자 너무 배고파서 길거리 음식도 하나 먹고\n여행기념 사진도 하나 찍고~\n드디어 포스팅해보는 인제여행기ㅋㅋㅋ 아 또가고싶다\nㅋㅋㅋㅋ남친이 좀 무리하긴 했지만 서로 연차 맞춰서 드디어 인제로 떠남\n강원도 인제 진짜 너무 좋았어요 ㅎㅎ', '오랜만에 숲길 걸으니 너무상쾌해서 좋았다.\n산책하면서 마음이 정화되는 기분이랄까~\n벼르고 벼르다 드디어 떠난 인제 여행..눈물난다ㅠㅠ\n\n여긴 진짜 힐링하러 가기 딱이에요! 강추!\n원대리 자작나무 숲 여행기 지금 시작합니다!']


In [26]:
#데이터 전처리_관련없는 게시글, 광고성 게시글 제거

#전처리될 키워드를 생성하여, 해당 키워드가 게시글에 2번이상 등장할 경우 삭제


#전처리 키워드 정의
stop_word = ['에어컨', '선풍기', '특가', '키즈', '통신사', '할인', '무료체험', '마스크', '마스크 팩', '100%', '영어학원','쿨링마스크팩',
            '쿨링 마스크 팩', '통신사', '물광', '혜택', '당첨', '1+1']

stop_post =[]
fill_post = []
cnt = []


for i in range(len(post_li)):
    cnt.append(sum([post_li[i].count(x) for x in stop_word]))


for post,cnt in zip(post_li,cnt):
    if cnt >=2 :
        stop_post.append(post)
    else:
        fill_post.append(post)


print(stop_post[:5])
print(f'결측 데이터 개수 : {len(stop_post)}')
print()

print(fill_post[:5])
print(f'전처리 후 남은 게시글 수 : {len(fill_post)}')

['영어학원 어디 찾으세요? 영어학원 고민은 인제 그만! \n 우리아이 초등교육 책임질 스피키 영어 학원\n통신사 단 3일간! 특가 이벤트 중입니다~ 서두르세요! 할인 기다려주신분들 할인혜택은 인제부터 시작입니다!\n\n통신사 단 3일간! 특가 이벤트 중입니다~ 서두르세요! 할인 기다려주신분들 할인혜택은 인제부터 시작입니다!\n📢 지금 가입만 해도 100% 에어컨 당첨 혜택🎉 지금 에어컨 받아가세요\n통신사 단 3일간! 특가 이벤트 중입니다~ 서두르세요! 할인 기다려주신분들 할인혜택은 인제부터 시작입니다!', '📢 지금 가입만 해도 100% 에어컨 당첨 혜택🎉 지금 에어컨 받아가세요\n통신사 단 3일간! 특가 이벤트 중입니다~ 서두르세요! 할인 기다려주신분들 할인혜택은 인제부터 시작입니다!\n📢 지금 가입만 해도 100% 에어컨 당첨 혜택🎉 지금 에어컨 받아가세요\n이 혜택이면 고민 끝이죠? 지금 바로 주문고고!\n\n📢 지금 가입만 해도 100% 에어컨 당첨 혜택🎉 지금 에어컨 받아가세요\n이 혜택이면 고민 끝이죠? 지금 바로 주문고고!\n✔️키즈 장난감 인기 상품 최저가 보장! 지금 바로 클릭\n무선 선풍기 무료체험 신청하면 사은품도 드려요😍 당첨기회\n이런 기회 인제 다시 없어요 ㅋㅋㅋ 물광 마스크 팩 지금 주문하면 1+1', '📢 지금 가입만 해도 100% 에어컨 당첨 혜택🎉 지금 에어컨 받아가세요\n무선 선풍기 무료체험 신청하면 사은품도 드려요😍 당첨기회\n\n📢 지금 가입만 해도 100% 에어컨 당첨 혜택🎉 지금 에어컨 받아가세요\n✔️키즈 장난감 인기 상품 최저가 보장! 지금 바로 클릭', '✔️키즈 장난감 인기 상품 최저가 보장! 지금 바로 클릭\n영어학원 어디 찾으세요? 영어학원 고민은 인제 그만! \n 우리아이 초등교육 책임질 스피키 영어 학원\n\n무선 선풍기 무료체험 신청하면 사은품도 드려요😍 당첨기회\n이런 기회 인제 다시 없어요 ㅋㅋㅋ 물광 마스크 팩 지금 주문하면 1+1\n📢 지금 가입만 해도 100% 에어컨 당첨 혜택🎉 지금 에어컨

In [27]:
#데이터 정규화 진행(+1차 불용어 제거)

for i in tqdm(range(len(fill_post)),total = len(fill_post), desc = '데이터 정규화 진행'):
    fill_post[i] = re.sub('\n', ' ', fill_post[i]) #줄바꿈 표시
    fill_post[i] = re.sub('[^A-Za-z0-9가-힣]',' ',fill_post[i]) #영어, 한글, 숫자만 남기기
    fill_post[i] = re.sub('존재하지 않는 이미지입니다', ' ', fill_post[i]) #블로그 사진 데이터
    fill_post[i] = re.sub('존재하지 않는 스티커입니다', ' ', fill_post[i]) #블로그 이모티콘 데이터
    fill_post[i] = re.sub('00:00|[0-9][0-9]:[0-9][0-9]', ' ', fill_post[i]) #영상재생
    fill_post[i] = re.sub('재생',' ',fill_post[i]) #영상재생
    fill_post[i] = re.sub('재생수|화질옵션', ' ', fill_post[i]) #영상재생
    fill_post[i] = repeat_normalize(fill_post[i], num_repeats = 2) # ㅋㅋㅋㅋ,ㅎㅎㅎㅎ => ㅋㅋ,ㅎㅎ로 남기기


#데이터 확인
print(fill_post[:10])

데이터 정규화 진행:   0%|          | 0/271 [00:00<?, ?it/s]

['난 언제 여행가나 했는데 드디어 왔습니다 강원도 인제 오랜만에 숲길 걸으니 너무상쾌해서 좋았다 드디어 포스팅해보는 인제여행기 아 또가고싶다 맑은 공기 마시니까 기분까지 맑아지더라구요', '난 언제 여행가나 했는데 드디어 왔습니다 강원도 인제 성수기에 와서 그런지 걱정했던것보단 아니지만 그래도 사람들이 꽤 많았음 다들 여름 휴가는 인제에 오기로 약속한거야 완전 가는 곳곳마다 사진 맛집 인생사진 겁나 건짐', '역시 사람은 도시가 아니라 자연속에 있어야해 매연없는 공기가 이렇게 좋다니 드디어 포스팅해보는 인제여행기 아 또가고싶다 성수기에 와서 그런지 걱정했던것보단 아니지만 그래도 사람들이 꽤 많았음 다들 여름 휴가는 인제에 오기로 약속한거야 자작나무 숲 사진보다 실물이 훨씬 예뻐요 자작나무 숲 사진보다 실물이 훨씬 예뻐요 성수기에 와서 그런지 걱정했던것보단 아니지만 그래도 사람들이 꽤 많았음 다들 여름 휴가는 인제에 오기로 약속한거야 벼르고 벼르다 드디어 떠난 인제 여행 눈물난다 여긴 진짜 힐링하러 가기 딱이에요 강추', '벼르고 벼르다 드디어 떠난 인제 여행 눈물난다 원대리 자작나무 숲 여행기 지금 시작합니다 도착하자마자 너무 배고파서 길거리 음식도 하나 먹고 여행기념 사진도 하나 찍고 드디어 포스팅해보는 인제여행기 아 또가고싶다 남친이 좀 무리하긴 했지만 서로 연차 맞춰서 드디어 인제로 떠남 강원도 인제 진짜 너무 좋았어요', '오랜만에 숲길 걸으니 너무상쾌해서 좋았다 산책하면서 마음이 정화되는 기분이랄까 벼르고 벼르다 드디어 떠난 인제 여행 눈물난다 여긴 진짜 힐링하러 가기 딱이에요 강추 원대리 자작나무 숲 여행기 지금 시작합니다', '오늘 소개할 여행지는 인제 원대리 자작나무 숲 남친이 좀 무리하긴 했지만 서로 연차 맞춰서 드디어 인제로 떠남 남친이 좀 무리하긴 했지만 서로 연차 맞춰서 드디어 인제로 떠남 도착하자마자 너무 배고파서 길거리 음식도 하나 먹고', '벼르고 벼르다 드디어 떠난 인제 여행 눈물난다 난 언제 여행가나 했는데 드디어 왔습니다 강원

In [28]:
#문장분리

import time

df1 = pd.DataFrame()

start = time.time()
kiwi = Kiwi()

split = []

for i in fill_post:
    split.append(kiwi.split_into_sents(i))

end = time.time()

print(f'걸린시간 : {end-start}')

df1['split_sentence'] = split

#문장분리 데이터 확인
df1

걸린시간 : 2.2712817192077637


,split_sentence
0,"[(난 언제 여행가나 했는데 드디어 왔습니다, 0, 22, None, []), (강..."
1,"[(난 언제 여행가나 했는데 드디어 왔습니다, 0, 22, None, []), (강..."
2,[(역시 사람은 도시가 아니라 자연속에 있어야해 매연없는 공기가 이렇게 좋다니 드디...
3,"[(벼르고 벼르다 드디어 떠난 인제 여행 눈물난다, 0, 25, None, []),..."
4,[(오랜만에 숲길 걸으니 너무상쾌해서 좋았다 산책하면서 마음이 정화되는 기분이랄까 ...
...,...
266,"[(인제에도 요즘 핫한 카페가 생겼어요, 0, 19, None, []), (분위기 ..."
267,[(인제 갬성 오지는 카페에 드디어 왔다규 커피도 존맛이라구 커플끼리 친구 끼리 가...
268,"[(당근 케이크 꼭 드세요, 0, 12, None, []), (사실 큰 기대 안하고..."
269,[(사실 큰 기대 안하고 주문한 시그니처 초콜릿 라떼가 너무 맛있어서 깜짝놀람 디저...


In [29]:
#kiwi로 문장분리한 데이터에서 실제 텍스트만 추출
#문장분리가 되었더라도 같은 게시글임을 표시하기 위해 같은 게시글끼리는 같은 인덱스 사용


df2 = pd.DataFrame()

for i in tqdm(range(len(df1['split_sentence'])), total = len(df1['split_sentence']), desc = '텍스트 추출'):
    for j in df1['split_sentence'][i]:
        di = {'index' : [i], 'text': j[0]} 
        split_df = pd.DataFrame(di)
        df2 = pd.concat([df2,split_df])

#데이터 확인
df2

텍스트 추출:   0%|          | 0/271 [00:00<?, ?it/s]

,index,text
0,0,난 언제 여행가나 했는데 드디어 왔습니다
0,0,강원도 인제 오랜만에 숲길 걸으니 너무상쾌해서 좋았다
0,0,드디어 포스팅해보는 인제여행기 아 또가고싶다 맑은 공기 마시니까 기분까지 맑아지더라구요
0,1,난 언제 여행가나 했는데 드디어 왔습니다
0,1,강원도 인제 성수기에 와서 그런지 걱정했던것보단 아니지만 그래도 사람들이 꽤 많았음
...,...,...
0,269,분위기 깡패 사실 큰 기대 안하고 주문한 시그니처 초콜릿 라떼가 너무 맛있어서 깜짝놀람
0,270,인제 마당있는 카페라서 산책하기 좋음
0,270,인제 마당있는 카페라서 산책하기 좋음
0,270,인제 마당있는 카페라서 산책하기 좋음


In [30]:
#문장분리 데이터 저장

df2.to_csv('가데이터_텍스트마이닝_문장분리.csv', encoding = 'utf-8')

In [31]:
#stem 처리로 졍규화된 텍스트에서 2차 불용어 처리


#불용어 사전 로드

with open('불용어 사전.txt', 'r', encoding = 'utf-8') as f:
    stop_word = list(set(f.read().splitlines()))

print(stop_word[:20])

['으로 인하여', '그치지 않다', '어쨋든', '이 정도의', '훨씬', '구', '초등교육', '할지라도', '비슷하다', '어째서', '끼리', '제', '봐', '하지 않는다면', '어떤것', '무슨', '남짓', '이르기까지', '수', '좀']


In [32]:
#형태소 분석


okt = Okt()

Noun= []
Verb = []
Adjective= []


for i in tqdm(df['text'], total = len(df['text']), desc = '형태소 분석 중'):
    pos_tag = okt.pos(i, stem = True)
    for word, pos in pos_tag:
        if pos == 'Noun' and word not in stop_word:
            Noun.append(word)
        elif pos == 'Verb' and word not in stop_word:
            Verb.append(word)
        elif pos == 'Adjective' and word not in stop_word:
            Adjective.append(word)


#데이터 확인
print(Noun[:15],'\n', Verb[:15],'\n', Adjective[:15])

형태소 분석 중:   0%|          | 0/300 [00:00<?, ?it/s]

['난', '여행가', '강원도', '인제', '숲길', '포스팅', '인제', '여행기', '맑은', '공기', '기분', '난', '여행가', '강원도', '인제'] 
 ['오다', '걸다', '해보다', '가다', '마시다', '오다', '오다', '들다', '가다', '겁나다', '건지다', '해보다', '가다', '오다', '들다'] 
 ['상쾌하다', '좋다', '맑다', '많다', '좋다', '많다', '예쁘다', '예쁘다', '많다', '배고프다', '무리하다', '좋다', '상쾌하다', '좋다', '무리하다']


In [33]:
#빈도 추출

Noun_cnt = Counter(Noun).most_common(len(Noun))
Verb_cnt = Counter(Verb).most_common(len(Verb))
Adjective_cnt = Counter(Adjective).most_common(len(Adjective))

print(Noun_cnt[:10], '\n', Verb_cnt[:10], '\n', Adjective_cnt[:10])

[('인제', 544), ('여행', 235), ('막국수', 188), ('맛집', 184), ('역시', 178), ('사진', 168), ('숲', 148), ('존재', 140), ('자작나무', 132), ('황태', 118)] 
 [('먹다', 311), ('오다', 268), ('가다', 158), ('않다', 140), ('떠나다', 121), ('받다', 109), ('찍다', 82), ('여기다', 81), ('주다', 70), ('들다', 56)] 
 [('좋다', 275), ('맛있다', 113), ('좋아하다', 96), ('야하다', 91), ('유명하다', 87), ('이렇다', 67), ('시원하다', 65), ('즐겁다', 61), ('많다', 55), ('고소하다', 52)]


In [34]:
#명사

n_df  = pd.DataFrame()

for i,j in Noun_cnt:
    di = {'단어' : [i], '빈도' : [j]}
    test_df = pd.DataFrame(di)
    n_df = pd.concat([n_df,test_df])


#1~100개 추출
n_df = n_df[:101]
n_df

,단어,빈도
0,인제,544
0,여행,235
0,막국수,188
0,맛집,184
0,역시,178
...,...,...
0,아시,27
0,아들,27
0,다시,26
0,물광,26


In [35]:
#동사

v_df = pd.DataFrame()

for i,j in Verb_cnt:
    di = {'단어' : [i], '빈도': [j]}
    test_df = pd.DataFrame(di)
    v_df = pd.concat([v_df,test_df])


#가데이터에서 동사는 불용어 처리후 100개가 안되어 전체 추출(전체 동사 개수 54개)
v_df

,단어,빈도
0,먹다,311
0,오다,268
0,가다,158
0,않다,140
0,떠나다,121
0,받다,109
0,찍다,82
0,여기다,81
0,주다,70
0,들다,56


In [36]:
#동사 v_df에사
#보다	52       보고오다	50   중복처리된 단어 합치기


# 1. '보고오다'의 빈도를 '보다'에 더함
v_df.loc[v_df['단어'] == '보다', '빈도'] += v_df.loc[v_df['단어'] == '보고오다', '빈도'].values[0]

# 2. '보고오다' 행 제거
v_df = v_df[v_df['단어'] != '보고오다']


v_df

,단어,빈도
0,먹다,311
0,오다,268
0,가다,158
0,않다,140
0,떠나다,121
0,받다,109
0,찍다,82
0,여기다,81
0,주다,70
0,들다,56


In [37]:
ad_df = pd.DataFrame()

for i,j in Adjective_cnt:
    di ={'단어' : [i], '빈도' : [j]}
    test_df = pd.DataFrame(di)
    ad_df = pd.concat([ad_df,test_df])

#가데이터에서 형용사는 불용어 처리후 100개가 안되어 전체 추출(전체 형용사 개수 32개)
ad_df

,단어,빈도
0,좋다,275
0,맛있다,113
0,좋아하다,96
0,야하다,91
0,유명하다,87
0,이렇다,67
0,시원하다,65
0,즐겁다,61
0,많다,55
0,고소하다,52


In [40]:
#단어 csv로 추출

n_df.to_csv('가데이터_텍스트마이닝_빈도(명사).csv', encoding = 'utf-8')
v_df.to_csv('가데이터_텍스트마이닝_빈도(동사).csv', encoding = 'utf-8')
ad_df.to_csv('가데이터_텍스트마이닝_빈도(형용사).csv', encoding = 'utf-8')

In [ ]:
##텍스트 마이닝
#2025.04.10 ~ 2025.05. 진행

#############################

#[진행목적]
#네이버에 키워드 '인제' 검색 후 나오는 4개년치 데이터 웹크롤링 진행
#웹크롤링을 통해 강원도 지역 인제와 인제 자작나무숲에 대한 트렌드 및 인식 파악


#[순서]
#빈도(워드클라우드 시각화) - TF-IDF - n-gram - 토픽모델링(LDA / bertpic) - concor분석(UCINET시각화) - 감정분석


#[참고]
#보안상 원본데이터 반출이 금지되어, 실제 데이터와 결과는 비슷 하지만 추론이 불가능한 가데이터로 다시 만든 코드
#실제 연구에서는 4개년치의 데이터를 사용했지만 데이터만 다를뿐 동일한 방식으로 연구를 진행하여 가데이터는 1년치 데이터만 만들어 진행
#데이터만 다를 뿐 실제 연구에 사용된 코드와 동일 코드 

In [19]:
#TF-IDF#

#TF-IDF도 원본데이터의 경우 문서별 명사,동사,형용사만 남기고 TF-IDf진행

In [97]:
#빈도분석에서 저장한 문장분리 데이터 가져오기


df = pd.read_csv('가데이터_텍스트마이닝_문장분리.csv', encoding = 'utf-8')
df

,Unnamed: 0,index,text
0,0,0,난 언제 여행가나 했는데 드디어 왔습니다
1,0,0,강원도 인제 오랜만에 숲길 걸으니 너무상쾌해서 좋았다
2,0,0,드디어 포스팅해보는 인제여행기 아 또가고싶다 맑은 공기 마시니까 기분까지 맑아지더라구요
3,0,1,난 언제 여행가나 했는데 드디어 왔습니다
4,0,1,강원도 인제 성수기에 와서 그런지 걱정했던것보단 아니지만 그래도 사람들이 꽤 많았음
...,...,...,...
1206,0,269,분위기 깡패 사실 큰 기대 안하고 주문한 시그니처 초콜릿 라떼가 너무 맛있어서 깜짝놀람
1207,0,270,인제 마당있는 카페라서 산책하기 좋음
1208,0,270,인제 마당있는 카페라서 산책하기 좋음
1209,0,270,인제 마당있는 카페라서 산책하기 좋음


In [98]:
#필요한 패키지 로드
import pandas as pd
import numpy as np

In [99]:
#데이터 정체
#TF-IDF는 데이터를 문자열만 받기때문에 혹시라도 문장분리데이터에 숫자만 남아있거나
#문자열이 아닌 데이터를 전부 문자열로 만들어주어야함

df = df['text'].astype('str').str.split()

In [100]:
df

0                           [난, 언제, 여행가나, 했는데, 드디어, 왔습니다]
1                   [강원도, 인제, 오랜만에, 숲길, 걸으니, 너무상쾌해서, 좋았다]
2       [드디어, 포스팅해보는, 인제여행기, 아, 또가고싶다, 맑은, 공기, 마시니까, 기...
3                           [난, 언제, 여행가나, 했는데, 드디어, 왔습니다]
4       [강원도, 인제, 성수기에, 와서, 그런지, 걱정했던것보단, 아니지만, 그래도, 사...
                              ...                        
1206    [분위기, 깡패, 사실, 큰, 기대, 안하고, 주문한, 시그니처, 초콜릿, 라떼가,...
1207                           [인제, 마당있는, 카페라서, 산책하기, 좋음]
1208                           [인제, 마당있는, 카페라서, 산책하기, 좋음]
1209                           [인제, 마당있는, 카페라서, 산책하기, 좋음]
1210    [인제, 애견카페, 추천, 강아지랑, 즐거운, 시간, 보내기, 조아용, 인제, 마당...
Name: text, Length: 1211, dtype: object

In [101]:
#데이터 리스트화

text = df.to_list()

print(text[:10])

[['난', '언제', '여행가나', '했는데', '드디어', '왔습니다'], ['강원도', '인제', '오랜만에', '숲길', '걸으니', '너무상쾌해서', '좋았다'], ['드디어', '포스팅해보는', '인제여행기', '아', '또가고싶다', '맑은', '공기', '마시니까', '기분까지', '맑아지더라구요'], ['난', '언제', '여행가나', '했는데', '드디어', '왔습니다'], ['강원도', '인제', '성수기에', '와서', '그런지', '걱정했던것보단', '아니지만', '그래도', '사람들이', '꽤', '많았음'], ['다들', '여름', '휴가는', '인제에', '오기로', '약속한거야'], ['완전', '가는', '곳곳마다', '사진', '맛집', '인생사진', '겁나', '건짐'], ['역시', '사람은', '도시가', '아니라', '자연속에', '있어야해', '매연없는', '공기가', '이렇게', '좋다니', '드디어', '포스팅해보는', '인제여행기', '아', '또가고싶다', '성수기에', '와서', '그런지', '걱정했던것보단', '아니지만', '그래도', '사람들이', '꽤', '많았음'], ['다들', '여름', '휴가는', '인제에', '오기로', '약속한거야'], ['자작나무', '숲', '사진보다', '실물이', '훨씬', '예뻐요']]


In [102]:
#데이터 불용어 제거

#불용어 사전 불러오기
with open('불용어 사전.txt', 'r', encoding = 'utf-8') as f:
    stop_word = list(set(f.read().splitlines()))

print(stop_word[:5])

['으로 인하여', '그치지 않다', '어쨋든', '이 정도의', '훨씬']


In [103]:
#불용어 제거 및 형태소 분석을 통한 정규화

from konlpy.tag import Okt
from tqdm.notebook import tqdm
okt=Okt()

fill_text = []

for i in tqdm(text, total = len(text), desc = '정규화 중'):
    fill = []
    for j in i:
        tag = okt.pos(j, stem = True)
        for word, pos in tag:
            if pos == 'Noun' and word not in stop_word:
                fill.append(word)
                fill_text.append(fill)
            elif pos == 'verb' and word not in stop_word:
                fill.append(word)
                fill_text.append(fill)
            elif pos == 'Adjective'and word not in stop_word:
                fill.append(word)
                fill_text.append(fill)
            else:
                pass

정규화 중:   0%|          | 0/1211 [00:00<?, ?it/s]

In [109]:
#데이터 확인
print(fill_text[:20])

[['여행가'], ['강원도', '인제', '숲길', '상쾌하다', '좋다'], ['강원도', '인제', '숲길', '상쾌하다', '좋다'], ['강원도', '인제', '숲길', '상쾌하다', '좋다'], ['강원도', '인제', '숲길', '상쾌하다', '좋다'], ['강원도', '인제', '숲길', '상쾌하다', '좋다'], ['인제', '여행기', '맑은', '공기', '기분', '맑다'], ['인제', '여행기', '맑은', '공기', '기분', '맑다'], ['인제', '여행기', '맑은', '공기', '기분', '맑다'], ['인제', '여행기', '맑은', '공기', '기분', '맑다'], ['인제', '여행기', '맑은', '공기', '기분', '맑다'], ['인제', '여행기', '맑은', '공기', '기분', '맑다'], ['여행가'], ['강원도', '인제', '걱정', '사람', '많다'], ['강원도', '인제', '걱정', '사람', '많다'], ['강원도', '인제', '걱정', '사람', '많다'], ['강원도', '인제', '걱정', '사람', '많다'], ['강원도', '인제', '걱정', '사람', '많다'], ['여름', '휴가', '인제', '오기'], ['여름', '휴가', '인제', '오기']]


In [111]:
#데이터 개별 리스트 제거하고 문자열만 남기기

text = [' '.join(x) for x in fill_text]

print(text[:10])

['여행가', '강원도 인제 숲길 상쾌하다 좋다', '강원도 인제 숲길 상쾌하다 좋다', '강원도 인제 숲길 상쾌하다 좋다', '강원도 인제 숲길 상쾌하다 좋다', '강원도 인제 숲길 상쾌하다 좋다', '인제 여행기 맑은 공기 기분 맑다', '인제 여행기 맑은 공기 기분 맑다', '인제 여행기 맑은 공기 기분 맑다', '인제 여행기 맑은 공기 기분 맑다']


In [125]:
#TF-IDF진행

from sklearn.feature_extraction.text import TfidfVectorizer

#객체 생성
vec = TfidfVectorizer()

#모델 학습
tfidf_matrix = vec.fit_transform(text)

#학습된 단어 추출
names = vec.get_feature_names_out()

#계산된 TF-IDf 희소행렬 1차원으로 변경
score = np.array(tfidf_matrix.sum(axis = 0)).flatten()

#단어 및 점수 확인
f_df = pd.DataFrame(
    {'단어' : names,
    'TF-IDF' : score} 
)

f_df

,단어,TF-IDF
0,가격,189.142492
1,가능,139.751726
2,가능하다,62.986211
3,가도,57.461964
4,가요,85.541527
...,...,...
185,현지,136.114839
186,황태,289.663588
187,후기,165.640128
188,휴가,72.420380


In [126]:
#데이터 점수별로 내림차순 정렬 후 100위 까지만 추출하기
f_df = f_df.sort_values('TF-IDF', ascending = False)

f_df[:101]

,단어,TF-IDF
135,인제,702.255949
153,좋다,521.104276
112,여행,496.077131
115,역시,429.070418
62,맛집,415.265998
...,...,...
79,분위기,90.129818
91,소개,89.276802
4,가요,85.541527
25,금방,85.541527


In [ ]:
#텍스트 마이닝
#2025.04.10 ~ 2025.05. 진행

#############################

#[진행목적]
#네이버에 키워드 '인제' 검색 후 나오는 4개년치 데이터 웹크롤링 진행
#웹크롤링을 통해 강원도 지역 인제와 인제 자작나무숲에 대한 트렌드 및 인식 파악


#[순서]
#빈도(워드클라우드 시각화) - TF-IDF - n-gram - 토픽모델링(LDA / bertpic) - concor분석(UCINET시각화) - 감정분석


#[참고]
#보안상 원본데이터 반출이 금지되어, 실제 데이터와 결과는 비슷 하지만 추론이 불가능한 가데이터로 다시 만든 코드
#실제 연구에서는 4개년치의 데이터를 사용했지만 데이터만 다를뿐 동일한 방식으로 연구를 진행하여 가데이터는 1년치 데이터만 만들어 진행
#데이터만 다를 뿐 실제 연구에 사용된 코드와 동일 코드 

In [ ]:
#사전활용 감정분석

#감정분석은 해당 게시글 자체의 감정을 나타낼 예정
#원본데이터에서는 부정감정 게시글도 존재했기때문에 이번 감정분석에서는 부정적인 게시글을
#원본데이터의 비율과 비슷하게 만들기 위해 45개의 부정 추가한 후 분석할 예정
#사전은 국립군산대학교의 KNU감정사전 활용

In [20]:
#부정게시글 만들기
import pandas as pd
import random




n_post = [
    # 강원도 인제 여행
    "길이 험해서 운전하는 데 힘들긴 했어요.\n몇 번 길도 잘못 들어갔는데 그래도 도착하니 공기는 참 좋더라고요.\n그 순간 피로가 좀 풀렸어요.",
    "생각보다 날씨가 흐려서 좀 아쉬웠어요.\n계획했던 트래킹은 못 했지만, 조용히 쉬기엔 딱 좋았어요.\n오히려 힐링이 된 것 같아요.",
    "관광지는 생각보다 사람이 없어서 좀 심심했어요.\n그래도 복잡하지 않아 한적하게 산책하기 좋았어요.\n마음은 편해졌어요.",
    "숙소 근처에 편의점이 없어서 불편했어요.\n조금 불편하긴 했지만 덕분에 더 자연과 가까워진 느낌이었어요.\n색다른 경험이었어요.",
    "가는 길에 차가 좀 막혀서 짜증 났는데,\n막상 도착하니 경치가 너무 좋아서 기분이 풀렸어요.\n사진도 많이 찍었네요.",
    "갑자기 비가 와서 우산도 없이 뛰어다녔어요.\n비 맞은 건 힘들었지만 나름 추억이 된 것 같아요.\n오히려 기억에 오래 남겠어요.",
    "음식점이 많지 않아 고르기 힘들었어요.\n그래도 우연히 들어간 곳이 맛집이어서 기분이 좋았어요.\n결국엔 대만족이었죠.",
    "혼자 여행이라 약간 외롭긴 했어요.\n그래도 마음 정리할 시간은 충분했어요.\n혼자만의 여행도 나쁘지 않네요.",
    "길이 미끄러워서 넘어질 뻔했어요.\n조심스럽게 다녔지만 그래도 경치 보며 걷는 재미는 있었어요.\n무사히 돌아와서 다행이에요.",
    "계획했던 스팟 몇 군데는 공사 중이더라고요.\n살짝 당황했지만 대신 근처에서 더 멋진 장소를 발견했어요.\n예상 밖의 발견이 좋았어요.",
    "새벽에 너무 추워서 고생했어요.\n따뜻한 커피 한 잔에 살짝 녹았어요.\n소소한 행복도 있더라고요.",

    # 인제카페
    "기다리는 줄이 너무 길어서 지쳤어요.\n그런데도 디저트 맛이 좋아서 기다린 보람 있었어요.\n예쁘게 사진도 찍었네요.",
    "커피 맛은 좀 밍밍했어요.\n그렇지만 카페 내부 분위기가 너무 좋아서 오래 앉아 있었네요.\n힐링하는 느낌이었어요.",
    "자리가 협소해서 불편했어요.\n그래도 창밖 풍경 보는 맛이 있어 잠시 잊게 됐네요.\n혼자 조용히 있기 딱 좋았어요.",
    "화장실은 좀 아쉬웠지만\n직원분이 정말 친절해서 기분이 좋아졌어요.\n사람이 분위기를 바꾸는 것 같아요.",
    "음료가 생각보다 달았어요.\n다음엔 덜 달게 요청해야겠어요.\n디저트랑은 잘 어울리긴 했네요.",
    "시끄러운 손님이 있어서 처음엔 좀 짜증났어요.\n그래도 나중엔 조용해져서 집중할 수 있었어요.\n결국엔 좋게 마무리됐어요.",
    "내부가 생각보다 어두웠어요.\n사진이 잘 안 나왔지만 분위기는 감성적이더라고요.\n은은한 조명이 좋았어요.",
    "기대했던 메뉴가 품절이라 아쉬웠어요.\n다른 메뉴 먹어봤는데 의외로 만족스러웠어요.\n새로운 걸 알아가는 재미도 있네요.",
    "카페가 외진 곳에 있어서 찾기 힘들었어요.\n그래도 그만큼 조용해서 좋았어요.\n숨은 보석 같은 느낌이었어요.",
    "처음에는 평범하다 느꼈지만,\n자세히 보니 인테리어 디테일이 꽤 감성적이었어요.\n재방문 의사 생겼네요.",
    "테이블 간격이 좁아서 조금 불편했어요.\n하지만 커피 향 덕분에 기분 좋게 머물렀어요.\n전반적으론 만족이에요.",

    # 가족여행
    "가족끼리 의견 충돌이 좀 있었어요.\n그래도 서로 양보하면서 좋은 시간 보냈어요.\n결국은 웃으며 마무리했네요.",
    "엄마 아빠가 불평이 많아서 초반엔 기운 빠졌어요.\n그래도 마지막 날엔 다들 만족한 얼굴이었어요.\n가족끼리 추억은 남았네요.",
    "아이들이 이동 중에 좀 힘들어했어요.\n하지만 도착해서 좋아하는 모습 보니 뿌듯했어요.\n기억에 남을 것 같아요.",
    "숙소가 생각보다 좁았어요.\n그래도 다 함께 붙어 있으니 더 웃긴 에피소드가 많았어요.\n오히려 즐거웠어요.",
    "계획대로 안 풀려서 초반엔 혼란스러웠어요.\n그래도 일정 바꿔서 편하게 다녔더니 오히려 더 좋았어요.",
    "애들이 너무 시끄럽게 떠들어서 살짝 지쳤어요.\n그만큼 신났다는 거겠죠.\n좋은 기억으로 남을 것 같아요.",
    "부모님이 체력이 안 따라줘서 일정 일부 생략했어요.\n대신 쉬는 시간이 많아져 여유로웠어요.\n피로도 풀리고 좋았어요.",
    "식당에서 음식이 늦게 나와 좀 짜증났지만,\n맛은 정말 괜찮았어요.\n결국 기분 좋게 나왔어요.",
    "계획이 빡빡해서 정신이 없었어요.\n그래도 다들 열심히 따라줘서 고마웠어요.\n다음엔 좀 여유롭게 다녀야겠어요.",
    "날씨가 흐려서 풍경은 덜했지만,\n오히려 사진이 감성적으로 나왔어요.\n뜻밖의 수확이었죠.",
    "돌발상황이 많았지만 다들 잘 대처해서\n결국엔 서로 더 돈독해진 기분이에요.\n가족여행은 역시 소중해요.",

    # 속초여행
    "속초에 사람 너무 많아서 정신이 없었어요.\n그래도 활기찬 분위기에 기분이 업됐네요.\n여행 느낌은 확실히 났어요.",
    "해변에서 바람이 너무 세서 머리가 다 엉켰어요.\n그래도 파도 소리 들으니 마음이 편해졌어요.\n역시 바다는 좋아요.",
    "계획한 맛집에 못 가서 아쉬웠어요.\n우연히 들어간 가게가 대박이었어요.\n결국 좋은 선택이었죠.",
    "주차가 힘들긴 했지만,\n도보로 다니면서 골목 구경하는 재미가 있었어요.\n걷는 여행도 나쁘지 않네요.",
    "숙소 뷰가 기대 이하였어요.\n그래도 청결하고 조용해서 편히 쉴 수 있었어요.\n푹 자고 나니 기분 좋았어요.",
    "장 보는 데 시간이 많이 걸렸어요.\n그래도 직접 요리해 먹으니 재미있었어요.\n가족과의 시간도 좋았고요.",
    "일출을 놓쳐서 너무 아쉬웠어요.\n하지만 대신 아침 산책하며 고요한 바다를 봤어요.\n그것도 참 좋았어요.",
    "시장 구경은 시끄럽고 복잡했어요.\n그래도 다양한 먹거리를 경험할 수 있었어요.\n속초다운 하루였죠.",
    "날씨가 추워서 오래 못 돌아다녔어요.\n카페 들어가 따뜻한 차 마시며 창밖 보니 힐링됐어요.",
    "원하는 기념품이 매진이라 아쉬웠어요.\n대신 독특한 로컬 제품을 샀는데 만족했어요.\n잘 찾아보면 보물 많네요."]


import re
from kiwipiepy import Kiwi
from tqdm.notebook import tqdm

#정규화
for i in range(len(n_post)):
    n_post[i] = re.sub('\n', ' ', n_post[i])
    n_post[i] = re.sub('[^A-Za-z0-9가-힣]', ' ', n_post[i])

print(n_post[:10],'\n')

['길이 험해서 운전하는 데 힘들긴 했어요  몇 번 길도 잘못 들어갔는데 그래도 도착하니 공기는 참 좋더라고요  그 순간 피로가 좀 풀렸어요 ', '생각보다 날씨가 흐려서 좀 아쉬웠어요  계획했던 트래킹은 못 했지만  조용히 쉬기엔 딱 좋았어요  오히려 힐링이 된 것 같아요 ', '관광지는 생각보다 사람이 없어서 좀 심심했어요  그래도 복잡하지 않아 한적하게 산책하기 좋았어요  마음은 편해졌어요 ', '숙소 근처에 편의점이 없어서 불편했어요  조금 불편하긴 했지만 덕분에 더 자연과 가까워진 느낌이었어요  색다른 경험이었어요 ', '가는 길에 차가 좀 막혀서 짜증 났는데  막상 도착하니 경치가 너무 좋아서 기분이 풀렸어요  사진도 많이 찍었네요 ', '갑자기 비가 와서 우산도 없이 뛰어다녔어요  비 맞은 건 힘들었지만 나름 추억이 된 것 같아요  오히려 기억에 오래 남겠어요 ', '음식점이 많지 않아 고르기 힘들었어요  그래도 우연히 들어간 곳이 맛집이어서 기분이 좋았어요  결국엔 대만족이었죠 ', '혼자 여행이라 약간 외롭긴 했어요  그래도 마음 정리할 시간은 충분했어요  혼자만의 여행도 나쁘지 않네요 ', '길이 미끄러워서 넘어질 뻔했어요  조심스럽게 다녔지만 그래도 경치 보며 걷는 재미는 있었어요  무사히 돌아와서 다행이에요 ', '계획했던 스팟 몇 군데는 공사 중이더라고요  살짝 당황했지만 대신 근처에서 더 멋진 장소를 발견했어요  예상 밖의 발견이 좋았어요 '] 



In [86]:
#빈도분석에서 저장한 문장분리 데이터 가져오기


df = pd.read_csv('가데이터_텍스트마이닝_문장분리.csv', encoding = 'utf-8')
df

,Unnamed: 0,index,text
0,0,0,난 언제 여행가나 했는데 드디어 왔습니다
1,0,0,강원도 인제 오랜만에 숲길 걸으니 너무상쾌해서 좋았다
2,0,0,드디어 포스팅해보는 인제여행기 아 또가고싶다 맑은 공기 마시니까 기분까지 맑아지더라구요
3,0,1,난 언제 여행가나 했는데 드디어 왔습니다
4,0,1,강원도 인제 성수기에 와서 그런지 걱정했던것보단 아니지만 그래도 사람들이 꽤 많았음
...,...,...,...
1206,0,269,분위기 깡패 사실 큰 기대 안하고 주문한 시그니처 초콜릿 라떼가 너무 맛있어서 깜짝놀람
1207,0,270,인제 마당있는 카페라서 산책하기 좋음
1208,0,270,인제 마당있는 카페라서 산책하기 좋음
1209,0,270,인제 마당있는 카페라서 산책하기 좋음


In [87]:
#감정사전 로드 후 딕셔너리에 저장

with open('pos_pol_word.txt', 'r', encoding = 'utf-8') as f:
    pos_word = list(set(f.read().splitlines()))

with open('neg_pol_word.txt', 'r', encoding = 'utf-8') as f:
    neg_word = list(set(f.read().splitlines()))


print(pos_word[:5])
print(neg_word[:5])
print()



#사전 정의
#긍정 : +1     #부정 -1
sen_dict = {}

for i in pos_word:
    sen_dict[i] = +1

for j in neg_word:
    sen_dict[j] = -1

# print(sen_dict) #사전확인

['', '후련한 모양', '조금 설레는', '표정이 환하다', '매끈하게']
['', '메스꺼워', '쳐부수는 일', '더럽게 여겨져', '(T_T)']



In [88]:
#문장분리된 게시글 하나로 합치기

group_df = df.groupby('index')['text'].apply(list).reset_index()

group_df

,index,text
0,0,"[난 언제 여행가나 했는데 드디어 왔습니다, 강원도 인제 오랜만에 숲길 걸으니 너무..."
1,1,"[난 언제 여행가나 했는데 드디어 왔습니다, 강원도 인제 성수기에 와서 그런지 걱정..."
2,2,[역시 사람은 도시가 아니라 자연속에 있어야해 매연없는 공기가 이렇게 좋다니 드디어...
3,3,"[벼르고 벼르다 드디어 떠난 인제 여행 눈물난다, 원대리 자작나무 숲 여행기 지금 ..."
4,4,[오랜만에 숲길 걸으니 너무상쾌해서 좋았다 산책하면서 마음이 정화되는 기분이랄까 벼...
...,...,...
266,266,"[인제에도 요즘 핫한 카페가 생겼어요, 분위기 깡패 인제 마당있는 카페라서 산책하기..."
267,267,[인제 갬성 오지는 카페에 드디어 왔다규 커피도 존맛이라구 커플끼리 친구 끼리 가족...
268,268,"[당근 케이크 꼭 드세요, 사실 큰 기대 안하고 주문한 시그니처 초콜릿 라떼가 너무..."
269,269,[사실 큰 기대 안하고 주문한 시그니처 초콜릿 라떼가 너무 맛있어서 깜짝놀람 디저트...


In [89]:
#부정게시글과 기존 게시글 합치기

n_post_df = pd.DataFrame(n_post, columns =['text'])
n_post_df['index'] = range(271,314)
n_post_df = n_post_df.groupby('index')['text'].apply(list).reset_index()

n_post_df

group_df = pd.concat([group_df,n_post_df])
group_df

,index,text
0,0,"[난 언제 여행가나 했는데 드디어 왔습니다, 강원도 인제 오랜만에 숲길 걸으니 너무..."
1,1,"[난 언제 여행가나 했는데 드디어 왔습니다, 강원도 인제 성수기에 와서 그런지 걱정..."
2,2,[역시 사람은 도시가 아니라 자연속에 있어야해 매연없는 공기가 이렇게 좋다니 드디어...
3,3,"[벼르고 벼르다 드디어 떠난 인제 여행 눈물난다, 원대리 자작나무 숲 여행기 지금 ..."
4,4,[오랜만에 숲길 걸으니 너무상쾌해서 좋았다 산책하면서 마음이 정화되는 기분이랄까 벼...
...,...,...
38,309,[장 보는 데 시간이 많이 걸렸어요 그래도 직접 요리해 먹으니 재미있었어요 가족...
39,310,[일출을 놓쳐서 너무 아쉬웠어요 하지만 대신 아침 산책하며 고요한 바다를 봤어요 ...
40,311,[시장 구경은 시끄럽고 복잡했어요 그래도 다양한 먹거리를 경험할 수 있었어요 속...
41,312,[날씨가 추워서 오래 못 돌아다녔어요 카페 들어가 따뜻한 차 마시며 창밖 보니 힐...


In [102]:
#불용어 사전 불러오기

with open('불용어 사전.txt','r', encoding = 'utf-8') as f:
    stop_word = list(set(f.read().splitlines()))

In [116]:
#감정분석

from konlpy.tag import Okt

okt = Okt()

result = []

for i,row in tqdm(group_df.iterrows(), desc = '감정분석 시작'):
    index = row['index']
    text = row['text']

    full_text = ' '.join(text)

    tag = okt.pos(full_text,stem = True)

    sen_score = 0
    matched_pos = []
    matched_neg = []


    for word,pos in tag:
        if pos in ['Noun', 'Verb', 'Adjective'] and word not in stop_word:
            if word in sen_dict:
                sen_score+= sen_dict[word]

                if sen_dict[word] > 0:
                    matched_pos.append(word)
                else:
                    matched_neg.append(word)
        
    total = len(matched_pos) + len(matched_neg)

    if total>0:
        positive = len(matched_pos) / total
        negative = len(matched_neg) / total
    else:
        positive = 0.0
        negative = 0.0

    if sen_score >0:
        sentiment = 'positive'
    elif sen_score < 0:
        sentiment = 'negative'
    else:
        sentiment = 'natural'


    result.append({
        'index' : index,
        'positive' : round(positive,2),
        'negative' : round(negative,2),
        'final_sentiment' : sentiment,
        'text' : full_text
    })

result_df = pd.DataFrame(result)

result_df

감정분석 시작: 0it [00:00, ?it/s]

,index,positive,negative,final_sentiment,text
0,0,1.00,0.00,positive,난 언제 여행가나 했는데 드디어 왔습니다 강원도 인제 오랜만에 숲길 걸으니 너무상쾌...
1,1,0.50,0.50,natural,난 언제 여행가나 했는데 드디어 왔습니다 강원도 인제 성수기에 와서 그런지 걱정했던...
2,2,0.75,0.25,positive,역시 사람은 도시가 아니라 자연속에 있어야해 매연없는 공기가 이렇게 좋다니 드디어 ...
3,3,0.50,0.50,natural,벼르고 벼르다 드디어 떠난 인제 여행 눈물난다 원대리 자작나무 숲 여행기 지금 시작...
4,4,1.00,0.00,positive,오랜만에 숲길 걸으니 너무상쾌해서 좋았다 산책하면서 마음이 정화되는 기분이랄까 벼르...
...,...,...,...,...,...
309,309,0.67,0.33,positive,장 보는 데 시간이 많이 걸렸어요 그래도 직접 요리해 먹으니 재미있었어요 가족과...
310,310,0.33,0.67,negative,일출을 놓쳐서 너무 아쉬웠어요 하지만 대신 아침 산책하며 고요한 바다를 봤어요 ...
311,311,0.33,0.67,negative,시장 구경은 시끄럽고 복잡했어요 그래도 다양한 먹거리를 경험할 수 있었어요 속초...
312,312,1.00,0.00,positive,날씨가 추워서 오래 못 돌아다녔어요 카페 들어가 따뜻한 차 마시며 창밖 보니 힐링...


In [117]:
#감정분석 결과 저장
result_df.to_csv('가데이터_텍스트마이닝_감정분석결과.csv',encoding = 'utf-8')

In [2]:
#텍스트 마이닝
#2025.04.10 ~ 2025.05. 진행

#############################

#[진행목적]
#네이버에 키워드 '인제' 검색 후 나오는 4개년치 데이터 웹크롤링 진행
#웹크롤링을 통해 강원도 지역 인제와 인제 자작나무숲에 대한 트렌드 및 인식 파악


#[순서]
#빈도(워드클라우드 시각화) - TF-IDF - n-gram - 토픽모델링(LDA / bertpic) - concor분석(UCINET시각화) - 감정분석


#[참고]
#보안상 원본데이터 반출이 금지되어, 실제 데이터와 결과는 비슷 하지만 추론이 불가능한 가데이터로 다시 만든 코드
#실제 연구에서는 4개년치의 데이터를 사용했지만 데이터만 다를뿐 동일한 방식으로 연구를 진행하여 가데이터는 1년치 데이터만 만들어 진행
#데이터만 다를 뿐 실제 연구에 사용된 코드와 동일 코드 

In [3]:
# concor분석

# 명사,동사,형용사만 추출 후, 3가지의 형태소를 전부 합친상태에서 빈도 기반 100개의 단어를 추출하여
# 빈도에 기반한 단어를 토대로 concor분석 예정
# 시각화는 UCINET 사용

In [41]:
import pandas as pd

#데이터 불러오기

df = pd.read_csv('가데이터_텍스트마이닝_문장분리.csv', encoding = 'utf-8')

df

,Unnamed: 0,index,text
0,0,0,난 언제 여행가나 했는데 드디어 왔습니다
1,0,0,강원도 인제 오랜만에 숲길 걸으니 너무상쾌해서 좋았다
2,0,0,드디어 포스팅해보는 인제여행기 아 또가고싶다 맑은 공기 마시니까 기분까지 맑아지더라구요
3,0,1,난 언제 여행가나 했는데 드디어 왔습니다
4,0,1,강원도 인제 성수기에 와서 그런지 걱정했던것보단 아니지만 그래도 사람들이 꽤 많았음
...,...,...,...
1206,0,269,분위기 깡패 사실 큰 기대 안하고 주문한 시그니처 초콜릿 라떼가 너무 맛있어서 깜짝놀람
1207,0,270,인제 마당있는 카페라서 산책하기 좋음
1208,0,270,인제 마당있는 카페라서 산책하기 좋음
1209,0,270,인제 마당있는 카페라서 산책하기 좋음


In [42]:
#불용어 처리 및 형태소 분석

#불용어 사전 불러오기
with open('불용어 사전.txt', 'r', encoding = 'utf-8') as f:
    stop_word = list(set(f.read().splitlines()))

print(stop_word[:10])

['영', '생각한대로', '잠깐', '아야', '아홉', '에 대해', '졸졸', '여전히', '누구', '이곳']


In [43]:
#블로그나 소셜미디어를 분석하는경우 문장 by 문장보다는 하나의 게시글 자체로
#concor분석을 하는게 더 맥락파악에 좋기때문에 인덱스별 같은 게시글로 그룹화


g_df = df.groupby('index')['text'].apply(list).reset_index()

g_df


#리스트 형태로 바꾸기
text = g_df['text'].to_list()

In [44]:
print(text[:3])

[['난 언제 여행가나 했는데 드디어 왔습니다', '강원도 인제 오랜만에 숲길 걸으니 너무상쾌해서 좋았다', '드디어 포스팅해보는 인제여행기 아 또가고싶다 맑은 공기 마시니까 기분까지 맑아지더라구요'], ['난 언제 여행가나 했는데 드디어 왔습니다', '강원도 인제 성수기에 와서 그런지 걱정했던것보단 아니지만 그래도 사람들이 꽤 많았음', '다들 여름 휴가는 인제에 오기로 약속한거야', '완전 가는 곳곳마다 사진 맛집 인생사진 겁나 건짐'], ['역시 사람은 도시가 아니라 자연속에 있어야해 매연없는 공기가 이렇게 좋다니 드디어 포스팅해보는 인제여행기 아 또가고싶다 성수기에 와서 그런지 걱정했던것보단 아니지만 그래도 사람들이 꽤 많았음', '다들 여름 휴가는 인제에 오기로 약속한거야', '자작나무 숲 사진보다 실물이 훨씬 예뻐요', '자작나무 숲 사진보다 실물이 훨씬 예뻐요', '성수기에 와서 그런지 걱정했던것보단 아니지만 그래도 사람들이 꽤 많았음', '다들 여름 휴가는 인제에 오기로 약속한거야 벼르고 벼르다 드디어 떠난 인제 여행 눈물난다', '여긴 진짜 힐링하러 가기 딱이에요', '강추']]


In [45]:

from tqdm.notebook import tqdm
from konlpy.tag import Okt
okt = Okt()

#형태소 분석을 통해 명사, 동사, 형용사만 남기기

token = []

for i in tqdm(text, desc = '형태소 분석'):
    full_text = ' '.join(i)
    tag = okt.pos(full_text, stem = True)
    tag_li = []
    for word,pos in tag:
        if pos in ['Noun', 'Verb', 'Adjective'] and word not in stop_word:
            tag_li.append(word)
            token.append(list(set(tag_li)))

#문서별로 명사, 동사, 형용사만 남은 상태
print(token[:5])

형태소 분석:   0%|          | 0/271 [00:00<?, ?it/s]

[['여행가'], ['여행가', '오다'], ['여행가', '오다', '강원도'], ['여행가', '인제', '오다', '강원도'], ['여행가', '강원도', '숲길', '인제', '오다']]


In [46]:
#token기준으로 빈도 상위100개 단어 추출


#단어 전체모음
# full_text = []

# for i in tqdm(token):
#     for j in i:
#         full_text.append(j)

# print(full_text[:50])
# print()


# from collections import Counter

# top_cnt = Counter(full_text).most_common(100)
# top_word = [i for i,j in top_cnt]

# print(top_word)

full_text = []

for i in tqdm(token):
    for j in i:
        full_text.append(j)

from collections import Counter
top_cnt = Counter(full_text).most_common(100)
top_word = [i for i, j in top_cnt]

  0%|          | 0/9904 [00:00<?, ?it/s]

In [47]:
#추출된 상위 100개와 동일한 단어만 남기기

fill_token = []

for i in tqdm(token):
    word = []
    for j in i:
        if j in top_word:
            word.append(j)
            fill_token.append(word)
        else:
            pass

print(fill_token[:10])

  0%|          | 0/9904 [00:00<?, ?it/s]

[['오다'], ['오다', '강원도'], ['오다', '강원도'], ['인제', '오다', '강원도'], ['인제', '오다', '강원도'], ['인제', '오다', '강원도'], ['강원도', '인제', '오다'], ['강원도', '인제', '오다'], ['강원도', '인제', '오다'], ['강원도', '인제', '오다']]


In [48]:
#공출현 쌍 빈도 추출하기

concor = {}

for word in tqdm(fill_token, desc = '공출현 쌍 빈도 추출'):
    for i,a in enumerate(word):
        for b in word[i+1:]:
            pair = tuple(sorted((a,b)))
            concor[pair] = concor.get(pair,0)+1
            

공출현 쌍 빈도 추출:   0%|          | 0/111070 [00:00<?, ?it/s]

In [49]:
print(concor)

{('강원도', '오다'): 5325, ('오다', '인제'): 52501, ('강원도', '인제'): 8959, ('강원도', '좋다'): 8738, ('인제', '좋다'): 32373, ('오다', '좋다'): 16922, ('좋다', '해보다'): 4119, ('강원도', '해보다'): 1753, ('인제', '해보다'): 5708, ('오다', '해보다'): 2954, ('여행기', '좋다'): 5417, ('강원도', '여행기'): 3409, ('여행기', '인제'): 8235, ('여행기', '해보다'): 5431, ('여행기', '오다'): 4059, ('가다', '여행기'): 5797, ('가다', '좋다'): 12465, ('가다', '강원도'): 3555, ('가다', '인제'): 26994, ('가다', '해보다'): 5315, ('가다', '오다'): 21536, ('공기', '여행기'): 4421, ('가다', '공기'): 4102, ('공기', '좋다'): 8788, ('강원도', '공기'): 3967, ('공기', '인제'): 8581, ('공기', '해보다'): 3140, ('공기', '오다'): 4544, ('기분', '여행기'): 2523, ('가다', '기분'): 2448, ('기분', '좋다'): 6024, ('강원도', '기분'): 4365, ('공기', '기분'): 6064, ('기분', '인제'): 6366, ('기분', '해보다'): 1583, ('기분', '오다'): 2446, ('강원도', '걱정'): 2279, ('걱정', '인제'): 6803, ('걱정', '오다'): 6967, ('강원도', '사람'): 4578, ('걱정', '사람'): 6863, ('사람', '인제'): 10357, ('사람', '오다'): 7995, ('강원도', '많다'): 3254, ('걱정', '많다'): 6659, ('많다', '인제'): 8580, ('많다', '사람'): 7240, ('많다', '오다'): 7348, ('들다'

In [50]:
#공출현 쌍 빈도 df로 만들기

import numpy 

con_matrix = pd.DataFrame([(a,b,c) for (a,b),c in concor.items()], columns = ['word1', 'word2', 'fre'])

con_matrix

,word1,word2,fre
0,강원도,오다,5325
1,오다,인제,52501
2,강원도,인제,8959
3,강원도,좋다,8738
4,인제,좋다,32373
...,...,...,...
2688,보다,여기다,230
2689,고소하다,애견,194
2690,맛집,애견,212
2691,고소하다,즐겁다,160


In [51]:
#상위 빈도 100개로추출한 단어로 공출현 행렬 생성

import numpy as np


con_fre = pd.DataFrame(np.zeros((100,100)), index =top_word, columns = top_word)

con_fre

,인제,오다,역시,맛집,좋다,먹다,여행,막국수,가다,황태,...,매연,무리하다,서로,존맛,해보다,오늘,눈물나다,걱정,하루,설악산
인제,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
오다,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
역시,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
맛집,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
좋다,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
오늘,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
눈물나다,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
걱정,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
하루,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
print(top_word)

['인제', '오다', '역시', '맛집', '좋다', '먹다', '여행', '막국수', '가다', '황태', '두부', '유명하다', '맛있다', '사진', '받다', '자작나무', '숲', '떠나다', '강원도', '여기다', '친구', '맛', '좋아하다', '주다', '감동', '시원하다', '카페', '인분', '국밥', '그릇', '낮다', '사람', '들다', '맥주', '고기', '이렇다', '남편', '많다', '추천', '육즙', '즐기다', '숨다', '녹음', '고소하다', '가격', '먹음', '생각', '착하다', '산책', '대박', '기분', '현지', '조합', '스스로', '주인', '바다', '찌다', '괜찮다', '비', '놀래다', '찍다', '할머니', '대리', '친절하다', '보고오다', '즐겁다', '오기', '공기', '속초', '성공', '대다', '여행기', '존', '가족', '보다', '인생', '여름', '캠핑', '힐링', '겨울', '놀다', '건지다', '예쁘다', '피곤하다', '도시', '남친', '아들', '연속', '애견', '춘천', '매연', '무리하다', '서로', '존맛', '해보다', '오늘', '눈물나다', '걱정', '하루', '설악산']


In [55]:
#공출현 matrix에 값 넣기

for a, b, c in con_matrix.values:
        con_fre.loc[a, b] = c
        con_fre.loc[b, a] = c


print(con_fre)

           인제       오다       역시       맛집       좋다       먹다       여행      막국수  \
인제        0.0  52501.0  46253.0  45113.0  32373.0  46069.0  30018.0  39849.0   
오다    52501.0      0.0  36547.0  37977.0  16922.0  37500.0  23632.0  37349.0   
역시    46253.0  36547.0      0.0  32081.0  21796.0  40489.0  19241.0  39046.0   
맛집    45113.0  37977.0  32081.0      0.0  21232.0  39974.0  24324.0  35813.0   
좋다    32373.0  16922.0  21796.0  21232.0      0.0  15871.0  12310.0  13458.0   
...       ...      ...      ...      ...      ...      ...      ...      ...   
오늘     3377.0   1209.0    478.0    720.0   1684.0    672.0   3680.0    514.0   
눈물나다   5695.0   1556.0   3146.0    817.0   3281.0   1035.0   5695.0      0.0   
걱정     6803.0   6967.0   3389.0   2204.0   3099.0    723.0   1585.0      0.0   
하루      597.0    396.0    902.0   3715.0   1296.0   1278.0   3498.0      0.0   
설악산       0.0      0.0    444.0   2120.0    897.0   1299.0   2257.0      0.0   

           가다       황태  ...      매연    